<a href="https://colab.research.google.com/github/ras1364/DetectFakeReviewsUsing-Sentence_BERT/blob/main/FakeReviewsWithSentenceEmbeddingUsingSBERT_Ver1_0_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.layers import TimeDistributed, GlobalAveragePooling1D, GlobalAveragePooling2D, BatchNormalization
from keras.layers import ELU, PReLU, LeakyReLU
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, AveragePooling1D
from keras.layers import Embedding
from keras.layers import Dropout, Flatten, Bidirectional, Dense, Activation, TimeDistributed
from keras.models import Model, Sequential
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from string import ascii_lowercase
from collections import Counter
# from gensim.models import Word2Vec
# from gensim.models import Doc2Vec
# from gensim.models import doc2vec
from gensim.models import KeyedVectors
import itertools, nltk, snowballstemmer, re

import tensorflow as tf
from tensorflow import keras 

In [3]:
data = pd.read_csv("/content/sample_data/deceptive-opinion.csv")
data.head()

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...


In [4]:
data['polarity'] = np.where(data['polarity']=='positive',1,0)
data['deceptive'] = np.where(data['deceptive']=='truthful', 1,0)
data.head()

,deceptive,hotel,polarity,source,text
0,1,conrad,1,TripAdvisor,We stayed for a one night getaway with family ...
1,1,hyatt,1,TripAdvisor,Triple A rate with upgrade to view room was le...
2,1,hyatt,1,TripAdvisor,This comes a little late as I'm finally catchi...
3,1,omni,1,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,1,hyatt,1,TripAdvisor,I asked for a high floor away from the elevato...


In [5]:
df = data.sample(frac=1)
df.head()

,deceptive,hotel,polarity,source,text
1194,1,intercontinental,0,Web,Stayed here in November 2011 on a leisure trip...
750,0,amalfi,1,MTurk,This hotel allowed me to experience the best o...
526,0,swissotel,1,MTurk,This is the hotel for the discriminating busin...
240,1,hilton,1,TripAdvisor,"went to chicago for a week in may, decided to ..."
1320,0,homewood,0,MTurk,I went to the Homewood Suites in Chicago which...


In [6]:
df.describe()

,deceptive,polarity
count,1600.000000,1600.000000
mean,0.500000,0.500000
std,0.500156,0.500156
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.500000,0.500000
75%,1.000000,1.000000
max,1.000000,1.000000


In [7]:
def create_class(c):
    if c['polarity'] == 1 and c['deceptive'] == 1:
        return [1,1]
    elif c['polarity'] == 1 and c['deceptive'] == 0:
        return [1,0]
    elif c['polarity'] == 0 and c['deceptive'] == 1:
        return [0,1]
    else:
        return [0,0]
    
def specific_class(c):
    if c['polarity'] == 1 and c['deceptive'] == 1:
        return "TRUE_POSITIVE"
    elif c['polarity'] == 1 and c['deceptive'] == 0:
        return "FALSE_POSITIVE"
    elif c['polarity'] == 0 and c['deceptive'] == 1:
        return "TRUE_NEGATIVE"
    else:
        return "FALSE_NEGATIVE"

data['final_class'] = data.apply(create_class, axis=1)
data['given_class'] = data.apply(specific_class, axis=1)
data['final_class']

0       [1, 1]
1       [1, 1]
2       [1, 1]
3       [1, 1]
4       [1, 1]
         ...  
1595    [0, 0]
1596    [0, 0]
1597    [0, 0]
1598    [0, 0]
1599    [0, 0]
Name: final_class, Length: 1600, dtype: object

In [8]:
data['given_class']

0        TRUE_POSITIVE
1        TRUE_POSITIVE
2        TRUE_POSITIVE
3        TRUE_POSITIVE
4        TRUE_POSITIVE
             ...      
1595    FALSE_NEGATIVE
1596    FALSE_NEGATIVE
1597    FALSE_NEGATIVE
1598    FALSE_NEGATIVE
1599    FALSE_NEGATIVE
Name: given_class, Length: 1600, dtype: object

In [9]:
data.head()

,deceptive,hotel,polarity,source,text,final_class,given_class
0,1,conrad,1,TripAdvisor,We stayed for a one night getaway with family ...,"[1, 1]",TRUE_POSITIVE
1,1,hyatt,1,TripAdvisor,Triple A rate with upgrade to view room was le...,"[1, 1]",TRUE_POSITIVE
2,1,hyatt,1,TripAdvisor,This comes a little late as I'm finally catchi...,"[1, 1]",TRUE_POSITIVE
3,1,omni,1,TripAdvisor,The Omni Chicago really delivers on all fronts...,"[1, 1]",TRUE_POSITIVE
4,1,hyatt,1,TripAdvisor,I asked for a high floor away from the elevato...,"[1, 1]",TRUE_POSITIVE


In [54]:
# Number of sentece in each review 
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
splitedReviews = []
for sen in data.text:
  data["NumberOfSentece"] = len(sent_tokenize(sen))
  # splitedReviews.append( sent_tokenize(sen))
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,deceptive,hotel,polarity,source,text,final_class,given_class,NumberOfSentece,stemmed_text_data,length
0,1,conrad,1,TripAdvisor,We stayed for a one night getaway with family ...,"[1, 1]",3,13,stay night getaway famili thursday tripl aaa r...,322
1,1,hyatt,1,TripAdvisor,Triple A rate with upgrade to view room was le...,"[1, 1]",3,13,tripl rate upgrad view room less $ includ brea...,180
2,1,hyatt,1,TripAdvisor,This comes a little late as I'm finally catchi...,"[1, 1]",3,13,come littl late final catch review past sever ...,614
3,1,omni,1,TripAdvisor,The Omni Chicago really delivers on all fronts...,"[1, 1]",3,13,omni chicago realli deliv front spacious room ...,385
4,1,hyatt,1,TripAdvisor,I asked for a high floor away from the elevato...,"[1, 1]",3,13,ask high floor away elev got room pleasant dec...,193


In [95]:
sum =0
for a in data.NumberOfSentece:
  sum += a
print("Average number of reviews : ", sum/len(data), "  Sum = ",sum , "  Length = ", len(data) )
print("Max number of sentece in review : ", max(data["NumberOfSentece"]))

Average number of reviews :  13.0   Sum =  20800   Length =  1600
Max number of sentece in review :  13
Average number of reviews :  13.0   Sum =  20800   Length =  1600
Max number of sentece in review :  13


In [56]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
data['given_class']= label_encoder.fit_transform(data['given_class']) 
data['given_class'].unique()

array([3, 1, 2, 0])

In [57]:
Y = data['given_class']
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
data.head()

,deceptive,hotel,polarity,source,text,final_class,given_class,NumberOfSentece,stemmed_text_data,length
0,1,conrad,1,TripAdvisor,We stayed for a one night getaway with family ...,"[1, 1]",3,13,stay night getaway famili thursday tripl aaa r...,322
1,1,hyatt,1,TripAdvisor,Triple A rate with upgrade to view room was le...,"[1, 1]",3,13,tripl rate upgrad view room less $ includ brea...,180
2,1,hyatt,1,TripAdvisor,This comes a little late as I'm finally catchi...,"[1, 1]",3,13,come littl late final catch review past sever ...,614
3,1,omni,1,TripAdvisor,The Omni Chicago really delivers on all fronts...,"[1, 1]",3,13,omni chicago realli deliv front spacious room ...,385
4,1,hyatt,1,TripAdvisor,I asked for a high floor away from the elevato...,"[1, 1]",3,13,ask high floor away elev got room pleasant dec...,193


In [58]:
textData = pd.DataFrame(list(data["text"])) # each row is one document; the raw text of the document should be in the 'text_data' column
textData.head()

,0
0,We stayed for a one night getaway with family ...
1,Triple A rate with upgrade to view room was le...
2,This comes a little late as I'm finally catchi...
3,The Omni Chicago really delivers on all fronts...
4,I asked for a high floor away from the elevato...


In [63]:
from gensim.parsing.preprocessing import remove_stopwords

def preprocess(text):
  return remove_stopwords(text)

data["stemmed_text_data"] = data["text"].map(preprocess)

In [64]:
data.head()

,deceptive,hotel,polarity,source,text,final_class,given_class,NumberOfSentece,stemmed_text_data,length
0,1,conrad,1,TripAdvisor,We stayed for a one night getaway with family ...,"[1, 1]",3,13,We stayed night getaway family thursday. Tripl...,322
1,1,hyatt,1,TripAdvisor,Triple A rate with upgrade to view room was le...,"[1, 1]",3,13,Triple A rate upgrade view room $200 included ...,180
2,1,hyatt,1,TripAdvisor,This comes a little late as I'm finally catchi...,"[1, 1]",3,13,This comes little late I'm finally catching re...,614
3,1,omni,1,TripAdvisor,The Omni Chicago really delivers on all fronts...,"[1, 1]",3,13,"The Omni Chicago delivers fronts, spaciousness...",385
4,1,hyatt,1,TripAdvisor,I asked for a high floor away from the elevato...,"[1, 1]",3,13,I asked high floor away elevator I got. The ro...,193


In [65]:
data["given_class"].value_counts()

3    400
1    400
2    400
0    400
Name: given_class, dtype: int64

In [66]:
data['length']=data['stemmed_text_data'].apply(len)
data.head()

,deceptive,hotel,polarity,source,text,final_class,given_class,NumberOfSentece,stemmed_text_data,length
0,1,conrad,1,TripAdvisor,We stayed for a one night getaway with family ...,"[1, 1]",3,13,We stayed night getaway family thursday. Tripl...,364
1,1,hyatt,1,TripAdvisor,Triple A rate with upgrade to view room was le...,"[1, 1]",3,13,Triple A rate upgrade view room $200 included ...,227
2,1,hyatt,1,TripAdvisor,This comes a little late as I'm finally catchi...,"[1, 1]",3,13,This comes little late I'm finally catching re...,714
3,1,omni,1,TripAdvisor,The Omni Chicago really delivers on all fronts...,"[1, 1]",3,13,"The Omni Chicago delivers fronts, spaciousness...",469
4,1,hyatt,1,TripAdvisor,I asked for a high floor away from the elevato...,"[1, 1]",3,13,I asked high floor away elevator I got. The ro...,231


In [67]:
print("The most words in a sentence = ",data.length.max())

The most words in a sentence =  2773


In [71]:
splitedReviews = []
for sentence in data.stemmed_text_data:
  splitedReviews.append(sent_tokenize(sentence))

In [23]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=704eac99fe5a92b1a544c7767d77a3515a0fbaec589b5ea952b2fd8890a09947
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [72]:
sentence_embeddings=[]
for row in splitedReviews:
    sentence_embeddings.append(sbert_model.encode(row,pad))#, show_progress_bar=True))
sentence_embeddings

[array([[ 0.0040611 , -0.0049281 , -0.01508004, ...,  0.01879989,
         -0.05758116, -0.01182375],
        [-0.07039636,  0.09348481, -0.0901555 , ..., -0.13046232,
         -0.06343081, -0.01312642],
        [-0.01047293, -0.06448124,  0.02742315, ...,  0.02544416,
         -0.11248863, -0.00714585],
        ...,
        [-0.06012868, -0.04458416,  0.03570691, ..., -0.12595224,
         -0.05461431,  0.02262034],
        [-0.00056172, -0.00257401,  0.02923678, ...,  0.0336418 ,
         -0.02969266, -0.07409082],
        [ 0.01607827,  0.02715304, -0.01489   , ..., -0.10371803,
         -0.10838132, -0.00438237]], dtype=float32),
 array([[-0.0376689 , -0.00668836, -0.0026207 , ..., -0.03233781,
         -0.10764214, -0.05220702],
        [ 0.04819376,  0.0132175 ,  0.03865111, ..., -0.03662772,
         -0.07045033, -0.02970263],
        [-0.05571179, -0.02581018, -0.0464279 , ..., -0.02327369,
          0.06663693, -0.00861282],
        [ 0.1308089 , -0.08068132,  0.07209074, ...,

In [78]:
from sklearn.model_selection import train_test_split
X = sentence_embeddings
y = data["deceptive"]


X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [79]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional, Dropout, BatchNormalization

In [80]:
from keras.preprocessing import sequence
from keras.layers import TimeDistributed, GlobalAveragePooling1D, GlobalAveragePooling2D, BatchNormalization
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, AveragePooling1D
#from keras.layers.embeddings import Embedding
from keras.layers import Dropout, Flatten, Bidirectional, Dense, Activation, TimeDistributed
from keras.models import Model, Sequential
from keras.utils import np_utils

In [ ]:
# maxlength = []
# for i in range(0,len(data.stemmed_text_data)):
#     maxlength.append(len(data.stemmed_text_data[i]))
    
# print(max(maxlength) )

2300


In [87]:
# X_train_new 
np.shape(X_train[])

(9, 384)

In [93]:
model=Sequential()
model.add(Conv1D(128, 5, activation='relu', input_shape=(13,384)))
model.add(BatchNormalization(renorm=True))
model.add(Dropout(0.25))
model.add(BatchNormalization(renorm=True))
model.add(MaxPool1D())
model.add(BatchNormalization(renorm=True))
model.add(Dropout(0.25))
model.add(BatchNormalization(renorm=True))

model.add(Conv1D(32, 5, activation='relu'))
model.add(BatchNormalization(renorm=True))
model.add(Dropout(0.25))
model.add(BatchNormalization(renorm=True))
model.add(MaxPool1D())
model.add(BatchNormalization(renorm=True))
model.add(Dropout(0.25))
model.add(BatchNormalization(renorm=True))

model.add(Conv1D(32, 5, activation='relu'))
model.add(BatchNormalization(renorm=True))
model.add(Dropout(0.25))
model.add(BatchNormalization(renorm=True))
model.add(MaxPool1D())
model.add(BatchNormalization(renorm=True))
model.add(Dropout(0.25))
model.add(BatchNormalization(renorm=True))

model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

ValueError: ignored

In [77]:
from keras.callbacks import ModelCheckpoint 
filename = 'weights.best.from_scratch%s.hdf5' % 9
checkpointer = ModelCheckpoint(filepath=filename, verbose=1, save_best_only=True)
model.fit(X_train, y_train, epochs=40, batch_size=512, callbacks=[checkpointer], validation_data=(X_test, y_test))


ValueError: ignored